# Decorating Classes

In [1]:
from fractions import Fraction
f = Fraction(2,3)
f.denominator

3

In [2]:
f.speak('hello')

AttributeError: 'Fraction' object has no attribute 'speak'

In [14]:
Fraction.speak = lambda self,message:f'Fraction says {message}'

print(f.speak('hello'))
f2 = Fraction(3,4)
f2.speak('hello world')
# add attribute dynamically at run-time.
# monkey patching.

Fraction says hello


'Fraction says hello world'

In [11]:
# we can add atrribute in a class
Fraction.is_integral = lambda self:self.denominator ==1

f1 = Fraction(2,3)
f2 = Fraction(8,1)

print(f1.is_integral())
print(f2.is_integral())

False
True


In [21]:
# we can add atrribute in a class using function / as decorator
def dec_speak(cls):
    cls.speak = lambda self,message:f'{self.__class__.__name__} says {message}'
    return cls

Fraction = dec_speak(Fraction)
f1 = Fraction(2,3)
print(f1.speak('jello'))

class Person:
    pass

Person = dec_speak(Person)
p = Person()
p.speak('i am a class')

Fraction says jello


'Person says i am a class'

# Example 0

In [1]:
def my_dec(a,b):
    def dec(fn):
        def inner(*args,**kwargs):
            print(f'decorated function called {a} {b}')
            return fn(*args,**kwargs)
        return inner
    return dec


@my_dec(10,20) # this is decorator factory.
def my_func(s):
    print('Hello',s)

my_func('World')

decorated function called 10 20
Hello World


In [4]:
class MyClass: # factory
    def __init__(self,a,b):
        self.a = a
        self.b = b
    
    def __call__(self,fn): # decorator
        def inner(*args,**kwargs):
            print(f'decorated function called {self.a} {self.b}')
            return fn(*args,**kwargs)
        return inner

@MyClass(10,20)
def my_func(s):
    print('Hello',s)

my_func("World")

decorated function called 10 20
Hello World


In [3]:
obj = MyClass(11,22)
my_func = obj(my_func)
my_func('World!')

decorated function called 11 22
decorated function called 10 20
Hello World!


# Example 1

In [34]:
from datetime import datetime,timezone

def info(self):
        results = []
        results.append(f'time: {datetime.now(timezone.utc)}')
        results.append(f'Class: {self.__class__.__name__}')
        results.append(f'id: {hex(id(self))}')
        for k,v in vars(self).items():
            results.append(f'{k}:{v}')
        return results


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Person:
    def __init__(self,name,dob):
        self.name = name
        self.dob = dob
    
    def say_hi(self):
        return 'Hello there'
    
p = Person('rabin',94)
p.debug()

['time: 2021-06-03 04:27:17.311628+00:00',
 'Class: Person',
 'id: 0x24dc168b490',
 'name:rabin',
 'dob:94']

In [43]:
@debug_info
class Automobile:
    def __init__(self,make,model,year,top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0
    
    @property
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self,new_speed):
        if new_speed>self.top_speed:
            raise ValueError('speed cannot exceed top_speed')
        else:
            self._speed = new_speed



favorite = Automobile('Ford','Model T',1908,45)
favorite.debug()

['time: 2021-06-03 04:32:52.960635+00:00',
 'Class: Automobile',
 'id: 0x24dc1756160',
 'make:Ford',
 'model:Model T',
 'year:1908',
 'top_speed:45',
 '_speed:0']

In [44]:
favorite.speed = 89

ValueError: speed cannot exceed top_speed

In [45]:
favorite.speed = 40
favorite.debug()

['time: 2021-06-03 04:33:39.503594+00:00',
 'Class: Automobile',
 'id: 0x24dc1756160',
 'make:Ford',
 'model:Model T',
 'year:1908',
 'top_speed:45',
 '_speed:40']

# Example 2

In [99]:

def complete_ordering(cls): # this is not the good idea. we can improve this.
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self,other: self < other or self == other
        cls.__gt__ = lambda self,other: not(self < other) and not(self == other)
        cls.__ne__ = lambda self,other: not(self == other)
    return cls

In [103]:
from math import sqrt
from functools import total_ordering

# @complete_ordering # it can be used but in python we already have better one.
@total_ordering # it just need is __eq__ and (__lt__ or __gt__)
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __eq__(self,other):
        if isinstance(other,Point):
            return (self.x ==other.x and self.y == other.y)
        else:
            return False

    def __lt__(self,other):
        if isinstance(other,Point):
            return abs(self)<abs(other)
        else:
            return NotImplemented
    
    def __repr__(self):
        return f'Point({self.x},{self.y})'
    
    # def __le__(self,other):
    # def __gt__(self,other):
    # def __ge__(self,other):
    # def __ne__(self,other):
    ##### we need to create the following methods. BUTTT we can use DECORATOR. so that every class can use it. by using equal and less_than.
    # a<=b iff a<b or a==b
    # a>b iff not(a<b) and a!=b
    # a>=b iff not (a<b)
    

p1,p2,p3 = Point(2,3),Point(2,3),Point(0,0)

In [104]:
print(abs(p1))
print(p1 is p2)
print(p1 == p2)
print(p3<p1)
print(p1>p3) # even if we have not implemented greater_then, python will look for if there is less_then python handle it.

3.605551275463989
False
True
True
True


In [105]:
print(p1>=p2)
print(p2<=p1)
print(p1!=p2)

True
True
False
